# Gezamenlijke, Marginale en Conditionele Kansen

In dit labo werken we met combinaties van random variabelen. We bestuderen gezamenlijke kansverdelingen, leren marginaliseren en berekenen conditionele kansen.

**Leerdoelen:**
- Gezamenlijke (joint) kansverdelingen implementeren en visualiseren
- Marginale verdelingen afleiden uit joint verdelingen
- Conditionele kansen berekenen en interpreteren
- Het concept van (on)afhankelijkheid begrijpen en toetsen

In [ ]:
import numpy as np

# Set random seed
rng = np.random.default_rng(42)

## Gezamenlijke Kansverdeling: Weer en Verplaatsingsmodus

Wanneer we de relatie tussen weertype en verplaatsingsmodus bekijken, hebben we te maken met twee random variabelen. De gezamenlijke verdeling beschrijft de kans op elke mogelijke combinatie.

**Scenario:**
- **X (Weertype):** Zonnig (1), Bewolkt (2), Regenachtig (3)
- **Y (Verplaatsingsmodus):** Te voet (1), Auto (2), Tram/Bus (3), Trein (4)

## ✍️
Construeer de joint probability table voor weertype en verplaatsingsmodus en beantwoord enkele kansberekening vragen.

**Probabiliteitswaarden (matrix representatie):**

|  | Te voet (y=1) | Auto (y=2) | Tram/Bus (y=3) | Trein (y=4) |
|---|---|---|---|---|
| **Zonnig (x=1)** | 0.20 | 0.10 | 0.12 | 0.08 |
| **Bewolkt (x=2)** | 0.10 | 0.08 | 0.08 | 0.04 |
| **Regenachtig (x=3)** | 0.03 | 0.07 | 0.07 | 0.03 |


## ✍️
Bereken de volgende kansen:
1. $P(x=1, y=1)$ - zonnig weer EN te voet
2. $P(x=3, y \in \{2,3\})$ - regenachtig EN (auto OF tram/bus)
3. $P(y=4)$ - trein (ongeacht het weer)

## Marginale Kansverdeling

De **marginale verdeling** van één variabele verkrijgen we door te sommeren over alle mogelijke waarden van de andere variabele(n).

Voor discrete variabelen: $P(x) = \sum_y P(x, y)$

## ✍️
Bereken de marginale verdelingen $P(x)$ en $P(y)$ uit de joint distributie.

## Conditionele Kansverdeling

De **conditionele kans** $P(y|x)$ geeft de kans op $y$ gegeven dat $x$ heeft plaatsgevonden:

$$P(y|x) = \frac{P(x, y)}{P(x)}$$

## ✍️
Bereken en visualiseer de conditionele verdeling $P(y|x=3)$: wat is de kans op elke verplaatsingsmodus, gegeven dat het regent?

## Continue Joint Distributions: Bivariate Gaussian

Voor continue variabelen gebruiken we joint PDFs. Een belangrijk voorbeeld is de bivariate Gauss-verdeling.

## ✍️
Visualiseer een bivariate normale verdeling met correlatie en bereken marginale/conditionele verdelingen.

## ✍️
Bereken en visualiseer de conditionele verdeling $p(x|y=y_0)$ voor een specifieke waarde van $y$.

## Onafhankelijkheid

Twee random variabelen zijn **onafhankelijk** als:
$$P(x, y) = P(x) \cdot P(y)$$

Equivalentie: als $x \perp y$, dan $P(x|y) = P(x)$

## ✍️
Vergelijk een bivariate verdeling met correlatie $\rho = 0$ (onafhankelijk) met $\rho = 0.7$ (afhankelijk).

## ✍️ Productregel

De productregel stelt dat elke joint distributie kan worden ontleed in conditionele distributies:

$$P(x, y) = P(y|x) \cdot P(x) = P(x|y) \cdot P(y)$$

Verifieer deze regel voor de weersomstandigheden x verplaatsingsmodus joint distributie.

## Voorbeeld: Afhankelijke variabelen

Laten we een voorbeeld bekijken waar variabelen duidelijk afhankelijk zijn.

## ✍️
Simuleer data waarbij $y = 2x + \epsilon$ met kleine ruis $\epsilon$. Visualiseer de joint, marginal en conditional distributies.